 **Start of the Sentimix Project**

**Imports**

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

**Device Check (GPU preferred)**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**Dataset Description**

Data info

The data is in CONLL format. It looks like:

meta uid sentiment

token lang_id  

meta uid sentiment

token lang_id

token lang_id 

**Uid** is a unique id for each tweet. 

**lang_id** is 'HIN' if the token is in Hindi, 'ENG' if the token is in English, and 'O' if the token is in neither of the languages. 

The **sentiment** is either positive, negative or neutral.

Total 17000 tweets are available.
The trial data has 1869 tweets.
The train data has 15131 additional tweets. (does not include trial data).
The train data after split has 14k tweets.
The validation data has 3k tweets.

**Upload the Data**

In [ ]:
from google.colab import files
!mkdir data
!mkdir Models
%cd /content/data

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘Models’: File exists
/content/data


Important : Note that if the upload below fails, then upload all the dataset files manually into the data folder created by previous commands. Then comment out the following code cells which contains the code for uploading files. Then Restart and Resun all cells.

Upload trial data (File name: trial_conll.txt)

In [ ]:
uploaded = files.upload()

Upload train set (Splitted train file) (File name: train_14k_split_conll.txt)

In [ ]:
uploaded = files.upload()

Upload development set (Splitted dev file) (File name: dev_3k_split_conll.txt)

In [ ]:
uploaded = files.upload()

Upload test set (File names: Hindi_test_unlabelled_conll_updated.txt, 	test_labels_hinglish.txt)

In [ ]:
uploaded = files.upload()
upload = files.upload()

**Changing Directory to Models**

In [ ]:
%cd /content/Models

/content/Models


**Importing library to hadle emojis**

In [ ]:
!pip install emoji
import emoji
from emoji import UNICODE_EMOJI

def is_emoji(s):
    return s in UNICODE_EMOJI
def emoji_to_text(s):
    return emoji.demojize(s)[1:-1]

**Create DataFrame of Training Data**

In [ ]:
tweet_id=[]
tweets=[]
temp=""
sentiment=[]
emojis=""
flag=False
file = open('/content/data/train_14k_split_conll.txt','r') 
for line in file: 
  if len(line.split())==3:
    if(flag):
      if (len(emojis)>1):
        temp=temp+" Emotions:"+' '.join(emojis.split('_'))
      tweets.append(temp)
      temp=""
      emojis=""
    flag=True
    tweet_id.append(int(line.split()[1]))
    sentiment.append(line.split()[2])
  elif len(line.split())==2:
    if (is_emoji(line.split()[0][0])):
      for i in range(len(line.split()[0])):
        emojis=emojis+" "+emoji_to_text(line.split()[0][i])
    else:
      temp=temp+" "+line.split()[0]  
if (len(emojis)>1):
  temp=temp+" Emotions:"+' '.join(emojis.split('_')) 
tweets.append(temp)
file.close()

temp=""
emojis=""
flag=False
file = open('/content/data/trial_conll.txt','r') 
for line in file: 
  if len(line.split())==3:
    if(flag):
      if (len(emojis)>1):
        temp=temp+" Emotions:"+' '.join(emojis.split('_'))
      tweets.append(temp)
      temp=""
      emojis=""
    flag=True
    tweet_id.append(int(line.split()[1]))
    sentiment.append(line.split()[2])
  elif len(line.split())==2:
    if (is_emoji(line.split()[0][0])):
      for i in range(len(line.split()[0])):
        emojis=emojis+" "+emoji_to_text(line.split()[0][i])
    else:
      temp=temp+" "+line.split()[0]  
if (len(emojis)>1):
  temp=temp+" Emotions:"+' '.join(emojis.split('_')) 
tweets.append(temp)
file.close()

df=pd.DataFrame()
df['id']=tweet_id
df['tweets']=tweets
df['sentiment']=sentiment
df.set_index('id', inplace=True)
df.head(20)


,tweets,sentiment
id,,
4330,nen á vist bolest vztek smutek zmatek osam ě ...,neutral
41616,@ nehantics Haan yaar neha kab karega woh pos...,neutral
6648,@ RahulGandhi television media congress ke li...,negative
2512,@ AmitShah @ narendramodi All India me nrc la...,positive
610,@ Nehr _ who @ TypoMantri @ anjanaomkashyap P...,neutral
14356,@ narendramodi jeet ki dher sari subh kamnaye...,positive
5840,@ Fakeionist @ SAMjhotaXpress @ PLiChapel but...,negative
32791,@ aajtak @ syedasimwaqar @ chitraaum @ syedza...,neutral
37480,@ TarekFatah Baih tere itjey Kya jalti hai Pa...,negative


**Create DataFrame of Validation Data**

In [ ]:
tweet_id=[]
tweets=[]
temp=""
sentiment=[]
emojis=""
flag=False
file = open('/content/data/dev_3k_split_conll.txt','r') 
for line in file: 
  if len(line.split())==3:
    if(flag):
      if (len(emojis)>1):
        temp=temp+" Emotions:"+' '.join(emojis.split('_'))
      tweets.append(temp)
      temp=""
      emojis=""
    flag=True
    tweet_id.append(int(line.split()[1]))
    sentiment.append(line.split()[2])
  elif len(line.split())==2:
    if (is_emoji(line.split()[0][0])):
      for i in range(len(line.split()[0])):
        emojis=emojis+" "+emoji_to_text(line.split()[0][i])
    else:
      temp=temp+" "+line.split()[0]  
if (len(emojis)>1):
  temp=temp+" Emotions:"+' '.join(emojis.split('_'))  
tweets.append(temp)
file.close()


df_val=pd.DataFrame()
df_val['id']=tweet_id
df_val['tweets']=tweets
df_val['sentiment']=sentiment
df_val.set_index('id', inplace=True)
df_val.head(20)

,tweets,sentiment
id,,
30258,@ prahladspatel modi mantrimandal may samil h...,positive
16648,@ bkunalraj @ TajinderBagga @ NikhilJakhar14 ...,negative
28511,@ waglenikhil U saw caste and religion in the...,negative
10466,@ DelhiPolice sir local police station pe com...,neutral
19266,Ve Maahi song from # Kesari is current favour...,positive
14547,@ jinhwitaek the only one boy that matters uh...,positive
22889,@ sarwatvalim Sarwatjee Wednesday ke Muqabil ...,positive
14328,RT @ Sunil _ Deodhar Today it was one of the ...,positive
24413,RT @ LalitJa21681145 Aapke shubh chintakk lal...,positive


**Create DataFrame of Test Data**

In [ ]:
tweet_id=[]
tweets=[]
temp=""
sentiment=[]
emojis=""
flag=False
file = open('/content/data/Hindi_test_unalbelled_conll_updated.txt','r') 
for line in file: 
  # print(line)
  if (len(line.split())==2 and line.split()[0]=="meta"):
    if(flag):  
      if (len(emojis)>1):
        temp=temp+" Emotions:"+' '.join(emojis.split('_'))
      tweets.append(temp)
      temp=""
      emojis=""
    flag=True
    tweet_id.append(line.split()[1])
  elif len(line.split())==2:
    if (is_emoji(line.split()[0][0])):
      for i in range(len(line.split()[0])):
        emojis=emojis+" "+emoji_to_text(line.split()[0][i])
    else:
      temp=temp+" "+line.split()[0]  
if (len(emojis)>1):
  temp=temp+" Emotions:"+' '.join(emojis.split('_')) 
tweets.append(temp)
file.close()

df_test=pd.read_csv('/content/data/test_labels_hinglish.txt')
df_test.rename(columns={'Uid':'id','Sentiment':'sentiment'}, inplace=True)
df_test.set_index('id', inplace=True)
df_test['tweets']=tweets
df_test.head(20)

,sentiment,tweets
id,,
20803,neutral,@ 454dkhan @ Heisunberg _ Agr kse ko itni imp...
20187,neutral,logon ko alloo pyaz tomator me toh allah pak ...
20953,neutral,@ LambaAlka Wafadaar bane rahane ka nayab tar...
13777,negative,@ varnishant @ narendramodi Chup bhosdike . H...
20990,positive,RT @ HardeepSPuri à¨¦à¨¾à¨¤à¨¾ à¨¸à©‹ à¨¸à¨¾à...
15075,neutral,@ bomanirani # HNY2 ki announcement hogyi to ...
32292,neutral,RT @ SharmakeF We're almost in the final thre...
4622,positive,@ 6thrat This song makes me super happy . I s...
22338,negative,@ sadifabi @ Shahidmasooddr tm dikha rhy ho a...


Counting The Number of Tweet sentiments

In [ ]:
df.sentiment.value_counts()

neutral     6018
positive    5216
negative    4635
Name: sentiment, dtype: int64

Giving labels to the sentiments

In [ ]:
possible_sentiments = df.sentiment.unique()

sentiment_dict = {}
for index, possible_sentiments in enumerate(possible_sentiments):
    sentiment_dict[possible_sentiments] = index

print(sentiment_dict)

df['label'] = df.sentiment.replace(sentiment_dict)
df.head(20)

df_val['label'] = df_val.sentiment.replace(sentiment_dict)
df_val.head(20)

df_test['label'] = df_test.sentiment.replace(sentiment_dict)
df_test.head(20)

{'neutral': 0, 'negative': 1, 'positive': 2}


,sentiment,tweets,label
id,,,
20803,neutral,@ 454dkhan @ Heisunberg _ Agr kse ko itni imp...,0
20187,neutral,logon ko alloo pyaz tomator me toh allah pak ...,0
20953,neutral,@ LambaAlka Wafadaar bane rahane ka nayab tar...,0
13777,negative,@ varnishant @ narendramodi Chup bhosdike . H...,1
20990,positive,RT @ HardeepSPuri à¨¦à¨¾à¨¤à¨¾ à¨¸à©‹ à¨¸à¨¾à...,2
15075,neutral,@ bomanirani # HNY2 ki announcement hogyi to ...,0
32292,neutral,RT @ SharmakeF We're almost in the final thre...,0
4622,positive,@ 6thrat This song makes me super happy . I s...,2
22338,negative,@ sadifabi @ Shahidmasooddr tm dikha rhy ho a...,1


**X_train and y_train**

In [ ]:
X_train, y_train = df.index.values, df.label.values

print(X_train[:25])
print(y_train[:25])

[ 4330 41616  6648  2512   610 14356  5840 32791 37480 16395 33428 19222
  8422 26930 40285 21403 35826 18017 39736 15893  5433 23492 13515 34678
 11310]
[0 0 1 2 0 2 1 0 1 0 0 0 0 2 2 2 2 1 2 1 1 2 2 0 0]


**X_val and y_val**

In [ ]:
X_val, y_val = df_val.index.values, df_val.label.values

print(X_val[:25])
print(y_val[:25])

[30258 16648 28511 10466 19266 14547 22889 14328 24413 25470  1362 31976
 33731 36215 27137   576 27534   569 28631 37864 13970 19296 33166  4537
 39079]
[2 1 1 0 2 2 2 2 2 2 2 2 1 0 0 0 0 2 2 2 1 2 2 1 2]


**X_test and y_test**

In [ ]:
X_test, y_test = df_test.index.values, df_test.label.values

print(X_test[:25])
print(y_test[:25])

[20803 20187 20953 13777 20990 15075 32292  4622 22338 23996 10426  3038
 37328 11313 32158  2598 12837 26050 30898 22656  2755 16061  3364  2926
 29391]
[0 0 0 1 2 0 0 2 1 2 2 1 1 0 2 0 0 1 2 2 1 1 0 2 0]


**Loading Tokenizer and Encoding our Data**

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', 
    do_lower_case=True
)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df.tweets.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

#For dev set
encoded_data_val = tokenizer.batch_encode_plus(
    df_val.tweets.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

#For test set
encoded_data_test = tokenizer.batch_encode_plus(
    df_test.tweets.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df.label.values)

#For dev set
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_val.label.values)

#For test set
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df_test.label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
len(dataset_train)

#For dev set
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
len(dataset_val)

#For test set
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)
len(dataset_test)

3000

**Setting up BERT Pretrained Model**

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(sentiment_dict),
    output_attentions=False,
    output_hidden_states=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**Creating Data Loaders**

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

#For dev set
dataloader_val = DataLoader(dataset_val, 
                            sampler=SequentialSampler(dataset_val), 
                            batch_size=batch_size)

#For test set
dataloader_test = DataLoader(dataset_test, 
                             sampler=SequentialSampler(dataset_test), 
                             batch_size=batch_size)

**Setting Up Optimiser and Scheduler**

Setting Number of Epochs

In [ ]:
epochs = 3

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

**Defining our Performance Metrics**

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')


def accuracy_per_class(preds, labels):
    total_numerator=0
    total_denominator=0
    label_dict_inverse = {v: k for k, v in sentiment_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        total_numerator=total_numerator+len(y_preds[y_preds==label])
        total_denominator=total_denominator+len(y_true)
    
    print(f'Total Accuracy: {(total_numerator/total_denominator)*100}\n')

**Mounting Drive to Save and Load Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH_FOR_MODEL_SAVE_AND_LOAD = "/content/Models/"
%cd /content/Models

/content/drive/My Drive/Google Colabs/Sentimix/Models


**Training Code**

In [ ]:
import random

seed_val = 12 #Random number setting for seed
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

**Training (Takes 1.5 hours for 3 Epochs)**

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'Training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.9302095112300688
Validation loss: 0.8562091335337213
F1 Score (Weighted): 0.5995280083434337



Epoch 2
Training loss: 0.8113338466613523
Validation loss: 0.8400179769130464
F1 Score (Weighted): 0.6000824745519523



Epoch 3
Training loss: 0.7339294647137004
Validation loss: 0.8314179549825952
F1 Score (Weighted): 0.6324169477752983



Epoch 4
Training loss: 0.6694973858614122
Validation loss: 0.8639342956086422
F1 Score (Weighted): 0.6238501645653779



Epoch 5
Training loss: 0.6308512117473348
Validation loss: 0.8547304903573179
F1 Score (Weighted): 0.6427776541614497



**Results from Validation set**

In [ ]:
_, predictions, true_vals = evaluate(dataloader_val)
val_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'F1 Score (Weighted): {val_f1}')
accuracy_per_class(predictions, true_vals)

F1 Score (Weighted): 0.6427776541614497
Class: neutral
Accuracy: 649/1128

Class: negative
Accuracy: 610/890

Class: positive
Accuracy: 669/982

Total Accuracy: 64.26666666666667



**Results from Test set**

In [ ]:
_, predictions, true_vals = evaluate(dataloader_test)
val_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'F1 Score (Weighted): {val_f1}')
accuracy_per_class(predictions, true_vals)

F1 Score (Weighted): 0.6913257453953022
Class: neutral
Accuracy: 736/1100

Class: negative
Accuracy: 606/900

Class: positive
Accuracy: 723/1000

Total Accuracy: 68.83333333333333



**Predicting Your Own Tweet**

In [ ]:
my_tweet = "Nai bhai, tmhara project amazing hai"

df=pd.DataFrame()
df['id']=[0]
df['tweets']=[my_tweet]
df.set_index('id', inplace=True)
df['label']=[0]


encoded_tweet = tokenizer.batch_encode_plus(
  df.tweets.values, 
  add_special_tokens=True, 
  return_attention_mask=True, 
  pad_to_max_length=True, 
  max_length=256, 
  return_tensors='pt'
)

input_ids = encoded_tweet['input_ids']
attention_masks = encoded_tweet['attention_mask']
labels = torch.tensor(df.label.values)
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, 
                        sampler=SequentialSampler(dataset), 
                        batch_size=batch_size)
_, predictions, true_vals = evaluate(dataloader)

label_dict_inverse = {v: k for k, v in sentiment_dict.items()}
mx = -1000
idx = 0
predictions = np.array(predictions[0])
predictions = list(predictions)
for i in range(len(predictions)):
  if(predictions[i]>mx):
    mx=predictions[i]
    idx=i
print(f'Tweet text: {my_tweet}')
print(f'Sentiment  : {label_dict_inverse[idx]}')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


Tweet text: Nai bhai, tmhara project amazing hai
Sentiment  : positive


**Using the saved Model**

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(sentiment_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load(PATH_FOR_MODEL_SAVE_AND_LOAD+'finetuned_BERT_epoch_3.model', map_location=torch.device('cuda'))) #Change epoch_3 to epoch_1 if you have set epochs = 1


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_test)
val_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'\nF1 Score (Weighted): {val_f1}')
accuracy_per_class(predictions, true_vals)


F1 Score (Weighted): 0.6919976680174325
Class: neutral
Accuracy: 711/1100

Class: negative
Accuracy: 642/900

Class: positive
Accuracy: 714/1000

Total Accuracy: 68.89999999999999



In [ ]:
my_tweet = "Complete the assignment nai toh fail ho jaaoge"

df=pd.DataFrame()
df['id']=[0]
df['tweets']=[my_tweet]
df.set_index('id', inplace=True)
df['label']=[0]


encoded_tweet = tokenizer.batch_encode_plus(
  df.tweets.values, 
  add_special_tokens=True, 
  return_attention_mask=True, 
  pad_to_max_length=True, 
  max_length=256, 
  return_tensors='pt'
)

input_ids = encoded_tweet['input_ids']
attention_masks = encoded_tweet['attention_mask']
labels = torch.tensor(df.label.values)
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, 
                        sampler=SequentialSampler(dataset), 
                        batch_size=batch_size)
_, predictions, true_vals = evaluate(dataloader)

label_dict_inverse = {v: k for k, v in sentiment_dict.items()}
mx = -1000
idx = 0
predictions = np.array(predictions[0])
predictions = list(predictions)
for i in range(len(predictions)):
  if(predictions[i]>mx):
    mx=predictions[i]
    idx=i
print(f'Tweet text: {my_tweet}')
print(f'Sentiment  : {label_dict_inverse[idx]}')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


Tweet text: Complete the assignment nai toh fail ho jaaoge
Sentiment  : neutral
